# **ALGORITMO GENETICO CON DEAP**
## Función Objetivo

La función objetivo que se optimizará es la siguiente:

$$
f(x) = (x^{2x}) - 1
$$


In [6]:
import random


# Función objetivo
def f(x):
    if x == 0:
        return 1
    return (x ** (2 * x)) - 1


# Función de evaluación que convierte un individuo (binario) en un número decimal
def evalOneMax(individual):
    numeroDecimal = int("".join(map(str, individual)), 2)
    return f(numeroDecimal)


# Función para inicializar una población de individuos con valores binarios (0 o 1)
def init_population(n_individuals, n_bits):
    return [[random.randint(0, 1) for _ in range(n_bits)] for _ in range(n_individuals)]


# Función para realizar la selección por torneo
def tournament_selection(population, fitnesses, k=3):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(list(zip(population, fitnesses)), k)
        winner = max(tournament, key=lambda x: x[1])
        selected.append(winner[0])
    return selected


# Función para realizar el cruce de dos puntos
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2


# Función para mutar individuos con una probabilidad 'mutpb'
def mutate(individual, mutpb=0.05):
    for i in range(len(individual)):
        if random.random() < mutpb:
            individual[i] = 1 - individual[i]  # Flip bit
    return individual


# Algoritmo genético
def genetic_algorithm(n_individuals, n_bits, cxpb, mutpb, ngen):
    # Inicializar la población
    population = init_population(n_individuals, n_bits)

    # Hall of Fame para guardar el mejor individuo
    hof = None
    best_fitness = float("-inf")

    for gen in range(ngen):
        # Evaluar el fitness de los individuos
        fitnesses = [evalOneMax(ind) for ind in population]

        # Guardar el mejor individuo
        for i, fit in enumerate(fitnesses):
            if fit > best_fitness:
                best_fitness = fit
                hof = population[i]

        # Mostrar estadísticas de la generación
        print(f"Generación {gen+1}: Mejor fitness: {best_fitness:.3e}")

        # Selección por torneo
        selected = tournament_selection(population, fitnesses)

        # Crear la siguiente generación
        next_population = []
        for i in range(0, len(selected), 2):
            parent1, parent2 = selected[i], selected[min(i + 1, len(selected) - 1)]
            if random.random() < cxpb:
                child1, child2 = crossover(parent1, parent2)
            else:
                child1, child2 = parent1, parent2
            next_population.extend([mutate(child1, mutpb), mutate(child2, mutpb)])

        # Reemplazar la población con la nueva generación
        population = next_population[:n_individuals]

    # Mostrar el mejor individuo
    print("\nMejor individuo:")
    print(f"Individuo: {hof}")
    print(f"Fitness del mejor individuo: {best_fitness:.3e}")


# Parámetros del algoritmo
n_individuals = 15  # Número de individuos
n_bits = 5  # Número de bits por individuo
cxpb = 0.5  # Probabilidad de cruce
mutpb = 0.2  # Probabilidad de mutación
ngen = 12  # Número de generaciones

# Ejecutar el algoritmo genético
genetic_algorithm(n_individuals, n_bits, cxpb, mutpb, ngen)


### En esta parte se coloca si se maximizará o minimizará

In [7]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximizar
creator.create("Individual", array.array, typecode="b", fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)


toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 5)  # Individuo con 5 bits
toolbox.register("population", tools.initRepeat, list, toolbox.individual)  # Población de individuos

# Función objetivo
def f(x):
    if x == 0:
        return 1
    return (x ** (2 * x)) - 1

# Función de evaluación
def evalOneMax(individual):
    numeroDecimal = int("".join(map(str, individual)), 2)
    return (f(numeroDecimal),)

# Registrar la función de evaluación
toolbox.register("evaluate", evalOneMax)

# Operadores genéticos
toolbox.register("mate", tools.cxTwoPoint)  
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  
toolbox.register("select", tools.selTournament, tournsize=3)

### Ejecución

In [8]:
def main():
    random.seed(60)

    pop = toolbox.population(n=15)
    hof = tools.HallOfFame(1) 
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(
        pop,
        toolbox,
        cxpb=0.5,  # Probabilidad de cruce
        mutpb=0.2,  # Probabilidad de mutación
        ngen=12,  # Número de generaciones
        stats=stats,
        halloffame=hof,
        verbose=True,
    )

    print("\nMejor individuo:")
    print(f"Individuo: {hof[0]}")
    print(f"Fitness del mejor individuo: {hof[0].fitness.values[0]}")

    return pop, log, hof

if __name__ == "__main__":
    main()

gen	nevals	avg        	std        	min	max        
0  	15    	7.32548e+79	2.74042e+80	15 	1.09863e+81
1  	5     	4.39607e+83	1.64457e+84	0  	6.59301e+84
2  	8     	2.82784e+87	1.05737e+88	8.14027e+22	4.23912e+88
3  	10    	3.88504e+91	9.90411e+91	1.77885e+66	2.91357e+92
4  	6     	9.71274e+91	1.37341e+92	6.59301e+84	2.91357e+92
5  	6     	1.94243e+92	1.37339e+92	6.59301e+84	2.91357e+92
6  	8     	2.91357e+92	0          	2.91357e+92	2.91357e+92
7  	7     	2.71933e+92	7.26771e+91	6.59301e+84	2.91357e+92
8  	11    	2.91357e+92	0          	2.91357e+92	2.91357e+92
9  	9     	2.91357e+92	0          	2.91357e+92	2.91357e+92
10 	6     	2.91357e+92	0          	2.91357e+92	2.91357e+92
11 	4     	2.91357e+92	0          	2.91357e+92	2.91357e+92
12 	8     	2.91357e+92	0          	2.91357e+92	2.91357e+92

Mejor individuo:
Individuo: Individual('b', [1, 1, 1, 1, 1])
Fitness del mejor individuo: 2.913567055049513e+92
